In [1]:
from crewai import Crew, Agent, Task,LLM
from crewai.tools import tool, BaseTool


In [2]:
@tool
def add(a: float, b: float) -> float:
    ''' This tool is used to add the numbers '''
    
    return a + b

In [3]:
@tool
def subtraction(a: float, b: float) -> float:
    ''' This tool is used to subtract the numbers '''
    
    return a - b

In [4]:
@tool
def multiplication(a: float, b: float) -> float:
    ''' This tool is used to multiply (*) the numbers '''
    
    return a * b

In [5]:
@tool
def division(a: float, b: float) -> float:
    ''' This tool is used divide  numbers where a is dividend and b is divisior'''
    
    
    return a / b

In [6]:
@tool
def modulus(a: float, b: float) -> float:
    ''' This tool is used perform modulus operation(%) between  numbers where a is dividend and b is divisior and returns remainder of a/b'''
    
    return a % b

In [7]:
'''Another way of making custom tools
from pydantic import Field, BaseModel
from typing import Tuple, Type

class MyToolInput(BaseModel):
    args: Tuple= Field(description= "The tuple containing multiple float(number)")

class sub_tool(BaseTool):
    name: str= 'Sub_tool'
    description: str= "This tool is used to subtract the numbers"
    args_schema: Type[BaseModel]= MyToolInput


    def _run(self, *args) -> float:
        self.result= 0
        for num in args:
            self.result= num - self.result

        return self.result
'''



'Another way of making custom tools\nfrom pydantic import Field, BaseModel\nfrom typing import Tuple, Type\n\nclass MyToolInput(BaseModel):\n    args: Tuple= Field(description= "The tuple containing multiple float(number)")\n\nclass sub_tool(BaseTool):\n    name: str= \'Sub_tool\'\n    description: str= "This tool is used to subtract the numbers"\n    args_schema: Type[BaseModel]= MyToolInput\n\n\n    def _run(self, *args) -> float:\n        self.result= 0\n        for num in args:\n            self.result= num - self.result\n\n        return self.result\n'

In [8]:
llm= LLM(
    model= 'huggingface/openai/gpt-oss-120b',
    temperature= 0.5
)

'''import os
from dotenv import load_dotenv
load_dotenv()
apikey= os.getenv("google2")
gemini_llm= LLM(
    model= "gemini/gemini-2.0-flash",
    api_key= apikey
   '''


llm2= LLM(
    model= 'huggingface/meta-llama/Llama-3.1-8B-Instruct'
)

In [9]:
calculator_agent = Agent(
    role="Calculator",
    goal="Solve math queries using tools",
    backstory='''You are a calculator ie simply a machine. You cannot do mathematical calculations of your own. But you are provided tools  that helps you solving mathematical problems and YOU MUST USE TOOLS PROVIDED TO YOU to solve the task. Without using tools you are always incorrect. You are purely dependent on tools.'
    The only rule you know is the sequence of execution of the operations ie BODMAS from left to right where
    B= Bracket,
    O= Of,
    D= Division,
    M= Multiplication,
    A= Addition and
    S= Subtraction.
    This will help you in solving the mathematical problems where there are multiple operations USING TOOLS''',
    llm=llm2,
    function_calling_llm=llm,
    allow_delegation= False,  
    tools=[add, subtraction, multiplication, division, modulus]
)


In [10]:
from pydantic import BaseModel, Field
class structured_output(BaseModel):
    Question: str= Field(description= "The given user question")
    Answer: float= Field(description= "The calculated answer")

In [11]:
task= Task(
    description= "This task is related for performing mathematical calculations on the basis of given user query: {query}. The agent MUST USE TOOLS to solve the query",
    expected_output= "The answer of given user mathematical problem: {query}",
    agent= calculator_agent,
    output_pydantic= structured_output,
    
    

)

In [12]:
calculator= Crew(
    agents= [calculator_agent],
    tasks= [task],
    output_log_file= 'log.txt',
    planning= True,
    planning_llm= llm,
    verbose= True,
    tracing= True
)

In [ ]:
result= calculator.kickoff({'query': ' (15+8)%7'})


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 132cb02f-7cb9-4f6c-81ae-58496e476f15                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


[2025-12-23 18:13:02][INFO]: Planning the crew execution


c:\Users\Yunish\miniconda3\envs\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Calculator                                                                                              │
│                                                                                                                 │
│  Task: This task is related for performing mathematical calculations on the basis of given user query:          │
│  (15+8)%7. The agent MUST USE TOOLS to solve the query1. **Understand the expression**: The user asks for the   │
│  result of (15+8)%7, which requires two operations – addition followed by modulus.                              │
│  2. **Select the appropriate tools**:                                                                           │
│     - Use the **add** tool to perform 15 + 8.                                                                   │
│     - Use the **modulus** tool to compute the remainder when the sum is divided by 7.                           │
│  3. **Execute the addition**:                                                                                   │
│     - Call the `add` tool with arguments `{ "a": 15, "b": 8 }`.                                                 │
│     - Capture the output; the tool returns `23`.                                                                │
│  4. **Execute the modulus operation**:                                                                          │
│     - Call the `modulus` tool with arguments `{ "a": 23, "b": 7 }`.                                             │
│     - Capture the output; the tool returns `2` (since 23 divided by 7 leaves a remainder of 2).                 │
│  5. **Validate the result**:                                                                                    │
│     - Verify that the intermediate addition result (23) is correct.                                             │
│     - Verify that the modulus result (2) matches manual calculation (23 % 7 = 2).                               │
│  6. **Return the final answer**:                                                                                │
│     - The Calculator agent should output the final result `2` as the answer to the user's query.                │
│  7. **Logging and reporting**:                                                                                  │
│     - Log each tool invocation with inputs and outputs for auditability.                                        │
│     - Provide a concise explanation to the user: "(15 + 8) = 23, and 23 % 7 = 2, so the answer is 2."           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 94268602-1421-4f1e-80b6-b48234aa810f                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Calculator                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "Question": "(15+8)%7",                                                                                      │
│    "Answer": 2                                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\Yunish\miniconda3\envs\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9bc571df-f5af-4427-b437-8d9244a31c52                                                                     │
│  Agent: Calculator                                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 132cb02f-7cb9-4f6c-81ae-58496e476f15                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "Question": "(15+8)%7",                                                                                      │
│    "Answer": 2                                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

┌───────────────────────── Trace Batch Finalization ──────────────────────────┐
│ ✅ Trace batch finalized with session ID:                                   │
│ b46c0859-0265-413c-a55b-1072cadbbd11                                        │
│                                                                             │
│ 🔗 View here:                                                               │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/b46c0859-0265-41 │
│ 3c-a55b-1072cadbbd11?access_code=TRACE-27012c30b9                           │
│ 🔑 Access Code: TRACE-27012c30b9                                            │
└─────────────────────────────────────────────────────────────────────────────┘
